In [1]:
## ODE
import numpy as np
import pandas as pd # for data manipulation
import time
from scipy.integrate import odeint, solve_ivp
from sklearn.metrics import mean_squared_error
import scipy.optimize as optimize

from ipynb.fs.full.myfun_model_usefulfuns import *
from ipynb.fs.defs.myfun_plot import *

2023-12-21 16:56:22.120730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 16:56:22.120759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 16:56:22.121808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 16:56:22.127473: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 16:56:22.928063: W tensorflow/compiler/tf2

# Lin and Log models

In [1]:
## Traffic dynamic with LWR-model

def TD_LWR_model(t, X, N, v0, L, flag):
    """
    Lighthill-Whitham-Richards (LWR) traffic flow model in 1D
    """
    #N, v0, L, flag = args[0], args[1], args[2], args[3]
    
    # W function
    match flag:
        case "Lin":
            W = lambda z: v0*max(1-1/z,0)
        case "Log":
            W = lambda z: v0*max(np.log(z),0)
        case _:
            return f"No match for {flag}, you can only choose between \"Lin\" and \"Log\""

    # ode sys
    d_x = np.zeros(N)
    
    for i in range(0,N-1):
        tmp = (X[i+1] - X[i])/L
        d_x[i] = W(tmp)

    d_x[N-1] = v0
        
    return d_x

## Optimize v0 and L in a scene

In [3]:
def define_loss(TD_LWR_model, flag, scn, v0, L, whole_tspan, x0, N, deltat):

    """
    f, loss_fun = define_loss(TD_LWR_model, flag, v0, L, whole_tspan, x0, N)
    """
    
    # Solve scn, freezing v0 and L
    def f(params):
        v0, L = params
        sol = odeint(TD_LWR_model, x0, whole_tspan, args=(N, v0, L, flag), tfirst = True).T  
        return sol

    # Define Loss function to be optimized
    def loss_fun(params):

        sol = f(params)
        _, sol_matched = match_timestamps_scene(whole_tspan, sol, deltat)
        return mean_squared_error(y_true = scn['Xarr'], y_pred = sol_matched)
    
    return f, loss_fun

In [4]:
def optimize_v0_L(TD_LWR_model, flag, scn, v0, L, deltat=0.05, method="L-BFGS-B", tol=1e-10):
    
    N, tstamps = scn['N. vehicles'], scn['Tarr']
    whole_tspan = time_discretization(tstamps[0], tstamps[-1], deltat)
    x0 = scn['Xarr'][:,0].tolist()
    
    f, loss_fun = define_loss(TD_LWR_model, flag, scn, v0, L, whole_tspan, x0, N, deltat)

    print('--'*50)
    print(f"Scene n.{scn.name}.\n\
    Optimizing parameters...\n\
    Initial mse: {loss_fun([v0,L])}\n")

    # Optimizing procedure
    bnds = ((0, np.inf), (2, np.inf))
    result = optimize.minimize(loss_fun, x0 = [v0,L], bounds = bnds, method=method,tol=tol)
    if result.success:
        v0_upd, L_upd = result.x
        print(f"After optimizing, new params: v0 = {v0_upd}, L = {L_upd}")
    else:
#          raise ValueError(result.message)
        v0_upd, L_upd = v0, L
        print("Optimization not succeed")
    

    print(f"\nmse: {loss_fun([v0_upd, L_upd])}")
    print('--'*50)
    
    return v0_upd, L_upd

## Ode solver in a scn

In [5]:
def solve_TD_LWR_scn(TD_LWR_model, flag, scn, v0, L, deltat, pplot=False):

    """
    x_list, t_list = solve_TD_LWR_scn(TD_LWR_model, flag, scn, v0, L, deltat)
    """
    
    N, tstamps, fmt = scn['N. vehicles'], scn['Tarr'], '{0:.02f}'
    
    print("--"*50)
    print(f"We have {len(tstamps)-1} time intervals inside \
    [{fmt.format(tstamps[0])},{fmt.format(tstamps[-1])}]")

    x_list, t_list, v_list = [[i] for i in scn['Xarr'][:,0]], [scn['Tarr'][0]], []

    for i in range(0,len(tstamps)-1):

        print(f"Time interval n.{i}: [{fmt.format(scn['Tarr'][i])}, {fmt.format(scn['Tarr'][i+1])}]")

        ## STEP 1: Solve the ODE sys in this time interval
        x0 = [l[-1] for l in np.vstack(x_list).tolist()]  # last values computed
        t0, tend = scn['Tarr'][i], scn['Tarr'][i+1]
        tspan = time_discretization(t0, tend, deltat=0.05)

        sol = odeint(TD_LWR_model, x0, tspan, args=(N, v0, L, flag), tfirst = True).T # transpose to get N trajs!

        ## STEP 2: store info
        x_list, t_list = update_sol_lists(N, tspan, sol, x_list, t_list)

        ## STEP 3: Plot the result: with scn['Xarr']
        if pplot:
            _, x_list_matched = match_timestamps_scene(t_list, x_list)
            title = rf"$TD\_LWR\_model\ using\ W\_{flag}$"
            plot_TD_LWR_scn(scn, x_list_matched, title)

    print("--"*50)
    
    return x_list, t_list

## Solve in a df (many scenes)

In [6]:
def solve_TD_LWR_df(df, TD_LWR_model, flag, v0=30, L=5, deltat=0.05, tol=1e-8, pplot=False):

    info_scn = []
    
    for _, scn in df.iterrows():

        tstamps, fmt = scn['Tarr'], '{0:.02f}'

        print("=="*50)
        print(f"df n.{df['N. file'][0]} - Scene n.{scn.name}/{len(df)}")

        # Optimize v0 and L
        v0_upd, L_upd = optimize_v0_L(TD_LWR_model, flag, scn, v0, L, deltat=0.05, method="L-BFGS-B", tol=1e-8)

        # Solve the ODE
        x_list, t_list = solve_TD_LWR_scn(TD_LWR_model, flag, scn, v0_upd, L_upd, deltat, pplot=False)
        
        # Store the result
        info_scn.append([t_list, x_list, v0_upd, L_upd, scn.name, df['N. file'][0]])

        print("=="*50)   

    # to better handle, transposte info_df
    tmp = list(map(list, zip(*info_scn)))

    info_df = pd.DataFrame({'t_list': tmp[0],\
                            'x_list': tmp[1],\
                            'v0_scn': tmp[2],\
                            'L_scn': tmp[3],\
                            'n_scn': tmp[4],\
                            'N. file': tmp[5]
                           })
  
    return info_df

In [7]:
def solve_TD_LWR_df_flag(df, TD_LWR_model, v0=30, L=5, deltat=0.05, tol=1e-8, pplot=False):
    
    """
    info_df = solve_TD_LWR_df_flag(df, TD_LWR_model, v0=30, L=5, deltat=0.05, tol=1e-8, pplot=False)
    
    Solve ODE for all the scenes in a df, using both Lin and then Log flag
    """
    
    l = []
    for flag in ['Lin', 'Log']:

        tmp = solve_TD_LWR_df(df, TD_LWR_model, flag, v0=30, L=5, deltat=0.05, tol=1e-12, pplot=False)
        tmp['LWR_flag'] = [flag] * tmp.shape[0]

        l.append(tmp)
        
    info_df = pd.concat(l)
    
    return info_df

## Solving in all the df

In [8]:
def solve_TD_LWR_dataset(dflist, TD_LWR_model, v0, L, deltat, tol, pplot=False):
    
    """
    info_alldataset = solve_TD_LWR_dataset(dflist, TD_LWR_model, v0=30, L=5, deltat=0.05, tol=1e-8, pplot=False)
    
    Solve ODE for all df (many scenes) in dflist, using both Lin and then Log flag
    """
    
    l = []
    for df in dflist:
        tmp = solve_TD_LWR_df_flag(df, TD_LWR_model, v0=30, L=5, deltat=0.05, tol=1e-8, pplot=False)    
        l.append(tmp)
        
    info_alldataset = pd.concat(l)
    
    return info_alldataset